<a href="https://colab.research.google.com/github/SONG-0502/1st-pap/blob/main/2%E9%82%BB%E5%9F%9F%E6%96%B9%E6%B3%95%EF%BC%88KNN%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

要实现基于 ml-100k 数据集的电影推荐系统，使用 邻域方法（KNN），并结合 特征工程（Feature Engineering），我们可以按照以下步骤来实现：

步骤 1：数据加载与预处理
加载数据并合并用户评分和电影信息。
清理数据，去除缺失值和重复值。
步骤 2：特征工程
在推荐系统中，特征工程是一个关键步骤。我们使用 Filter、Wrapper 和 Embedded 方法来提取特征：

Filter 方法：通过对数据进行筛选，去除评分极少的用户和电影，确保数据集更加均衡。
Wrapper 方法：使用 KNN 算法来构建基于用户或电影相似度的邻域推荐系统。
Embedded 方法：通过计算用户或电影的相似度矩阵，直接嵌入到 KNN 算法中。
步骤 3：使用 KNN 计算相似度
计算用户间或电影间的相似度，选择最近邻（top-N）进行推荐。
步骤 4：模型评估
使用 RMSE 或 Precision@K 等指标来评估模型的效果。
步骤 5：保存模型
使用 pickle 将训练好的 KNN 模型保存为 pkl 文件，方便后续使用。

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
import pickle

# 1. 数据加载与预处理
column_names = ['user_id', 'movie_id', 'rating', 'timestamp']
data = pd.read_csv('u.data', sep='\t', names=column_names)

# 加载电影信息
movie_column_names = ['movie_id', 'title']
movies = pd.read_csv('u.item', sep='|', encoding='latin-1', names=movie_column_names, usecols=[0, 1])

# 合并评分数据与电影信息
data = pd.merge(data, movies, on='movie_id')

# 数据清洗
# 删除重复的用户评分
data.drop_duplicates(inplace=True)

# 查看数据
print(data.head())

# 2. 特征工程：创建用户-物品评分矩阵（R矩阵）
R_df = data.pivot(index='user_id', columns='movie_id', values='rating')
R = R_df.fillna(0).values  # 将缺失值填充为0

# 对每个用户的评分进行均值归一化（Filter方法）
user_ratings_mean = np.mean(R, axis=1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

# 将数据集分为训练集和测试集（Wrapper方法）
train_data, test_data = train_test_split(R_demeaned, test_size=0.2, random_state=42)

# 3. 使用KNN进行推荐：计算用户或电影的相似度
# 使用cosine similarity计算电影之间的相似度矩阵
cosine_sim = cosine_similarity(R.T)  # 转置后按电影计算相似度
print("Cosine similarity matrix shape:", cosine_sim.shape)

# 4. 为每个用户进行推荐：根据电影的相似度
def recommend_movies(user_id, R, cosine_sim, top_n=10):
    # 获取该用户的评分向量
    user_ratings = R[user_id - 1]  # user_id从1开始，所以减1
    similar_scores = cosine_sim.dot(user_ratings)

    # 获取前top_n的电影
    similar_movie_indices = similar_scores.argsort()[-top_n:][::-1]
    return similar_movie_indices

# 推荐给用户1的电影
user_id = 1
recommended_movie_indices = recommend_movies(user_id, R, cosine_sim, top_n=10)

# 打印推荐的电影
recommended_movies = movies.iloc[recommended_movie_indices]
print(f"Top 10 movie recommendations for user {user_id}:")
print(recommended_movies)

# 5. 评估模型：计算RMSE（均方根误差）
def rmse(predictions, targets):
    # 只计算评分大于0的部分
    non_zero_indices = targets > 0  # 仅计算用户评分过的电影
    return np.sqrt(mean_squared_error(targets[non_zero_indices], predictions[non_zero_indices]))

# 使用测试数据计算RMSE
test_predictions = cosine_sim.dot(test_data.T).T  # Changed the order and transposed
test_rmse = rmse(test_predictions, test_data)
print("RMSE on the test data:", test_rmse)

# 6. 保存模型（pkl）
model = {
    'cosine_sim': cosine_sim,
    'user_ratings_mean': user_ratings_mean
}

# 保存模型
with open('knn_recommendation_model.pkl', 'wb') as f:
    pickle.dump(model, f)

print("Model saved as knn_recommendation_model.pkl")

# 7. 加载模型并进行推荐
with open('knn_recommendation_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

# 提取加载的相似度矩阵
cosine_sim_loaded = loaded_model['cosine_sim']
user_ratings_mean_loaded = loaded_model['user_ratings_mean']

# 使用加载的模型推荐电影
recommended_movie_indices_loaded = recommend_movies(user_id, R, cosine_sim_loaded, top_n=10)
recommended_movies_loaded = movies.iloc[recommended_movie_indices_loaded]

print(f"Top 10 movie recommendations for user {user_id} (after loading the model):")
print(recommended_movies_loaded)


   user_id  movie_id  rating  timestamp                       title
0      196       242       3  881250949                Kolya (1996)
1      186       302       3  891717742    L.A. Confidential (1997)
2       22       377       1  878887116         Heavyweights (1994)
3      244        51       2  880606923  Legends of the Fall (1994)
4      166       346       1  886397596         Jackie Brown (1997)
Cosine similarity matrix shape: (1682, 1682)
Top 10 movie recommendations for user 1:
     movie_id                                      title
173       174             Raiders of the Lost Ark (1981)
203       204                  Back to the Future (1985)
171       172            Empire Strikes Back, The (1980)
55         56                        Pulp Fiction (1994)
167       168     Monty Python and the Holy Grail (1974)
97         98           Silence of the Lambs, The (1991)
49         50                           Star Wars (1977)
194       195                     Terminator, The 

代码解释
1. 数据加载与预处理

加载数据：通过 pd.read_csv 加载了 u.data（包含用户评分）和 u.item（包含电影信息）文件。
数据合并：使用 pd.merge() 将评分数据与电影信息合并。
数据清洗：删除重复评分，确保数据一致性。
2. 特征工程（Filter、Wrapper、Embedded）

Filter方法：通过对评分矩阵进行 均值归一化，去除每个用户的评分偏差，避免评分普遍偏高或偏低对推荐系统产生影响。
Wrapper方法：使用 KNN（k-Nearest Neighbors） 计算相似度，选择与目标电影/用户最相似的邻居进行推荐。
Embedded方法：通过计算电影间的 余弦相似度，直接嵌入到 KNN 模型中进行电影推荐。
3. 使用KNN计算相似度

计算相似度：通过 余弦相似度（cosine_similarity）计算电影之间的相似度矩阵。余弦相似度度量了电影评分向量之间的角度差异，值越大表示越相似。
4. 为每个用户进行推荐

对于每个用户，获取该用户对电影的评分向量，然后通过与所有其他电影的相似度矩阵计算加权评分。推荐得分最高的 top_n 部电影作为推荐结果。
5. 模型评估（RMSE）

使用 RMSE（均方根误差） 来评估模型的预测效果，特别是对测试集的预测结果。我们只计算评分大于0的部分（用户已评分的电影）。
6. 保存与加载模型

使用 pickle 将训练好的相似度矩阵和用户评分均值保存为 knn_recommendation_model.pkl 文件。加载后，我们可以继续使用该模型来进行推荐。
7. 推荐并输出结果

根据用户的评分历史，推荐最相似的电影，并输出推荐列表。
结果：
我们通过 KNN 方法计算电影之间的相似度，并为每个用户推荐最相似的电影。
评估了模型的性能，计算了 RMSE 来判断模型的效果。
通过 pickle 保存并加载模型，能够确保模型的持久化，便于后续使用。
总结
本项目利用了 KNN（邻域方法） 结合 特征工程 提取特征，构建了一个基于相似度的电影推荐系统。
通过 余弦相似度 计算电影之间的相似性，并使用 RMSE 进行模型评估。
最后通过 pickle 将训练好的模型保存为 pkl 文件，便于后续加载使用。
你可以根据实际需求调整推荐算法中的参数，例如邻居数量 k，以及评估方式等。